# GCN in a transductive setting for Node betweenness
This notebook shows how a GCN/GraphSAGE model is trained to compute Node betweenness centrality on different graphs

# 1. Codebase

In [19]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
import torch.nn as nn

class MyNet():

    def __str__(self):
        #print(dir(self))
        #print()
        #print([attr+"-"+str(getattr(self,attr)) for attr in dir(self) if attr.startswith("d")])
        hp = [attr+"="+str(getattr(self,attr)) for attr in dir(self) if attr.startswith("d") and len(attr)==2]
        hp = "_".join(hp)
        return "%s-%s" % (self.__class__.__name__,hp)
        #return "%s-gcn(%d,%d)-gcn(%d,%d)" % (self.__class__.__name__,dataset.num_features,self.d1,self.d1,
        #                                       dataset.num_classes)
        
        
class Net1(torch.nn.Module, MyNet):
    def __init__(self, d1=16,num_features=1, num_classes=1):
        super(Net1, self).__init__()
        self.conv1 = GCNConv(num_features, d1)
        self.conv2 = GCNConv(d1, num_classes)
        self.d1 = d1
        

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        # output as multiclass target
        #return F.log_softmax(x, dim=1)
        
        # output as regression target
        return x
    


    
class Net2(torch.nn.Module, MyNet):
    def __init__(self, d1=300,d2=100,num_features=1, num_classes=1):
        super(Net2, self).__init__()
        self.conv1 = GCNConv(num_features, d1 )
        #self.conv2 = GCNConv(16, dataset.num_classes)
        self.fc1 = nn.Linear(d1, d2)
        self.fc2 = nn.Linear(d2, num_features)
        self.d1 = d1
        self.d2 = d2

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # 2 fc layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # output as regression target
        return x
        
    
    #def __str__(self):
    #    return "%s-gcn(%d,%d)-fc(%d,%d)-fc(%d,%d)" % (self.__class__.__name__,dataset.num_features,self.d1,self.d1,
    #                                                    self.d2,self.d2,
    #                                                    dataset.num_features)

    
    
class Net3(torch.nn.Module, MyNet):
    def __init__(self, d1=300,d2=100,num_features=1, num_classes=1):
        super(Net3, self).__init__()
        self.conv1 = SAGEConv(num_features, d1 )
        #self.conv2 = SAGEConv(16, dataset.num_classes)
        self.fc1 = nn.Linear(d1, d2)
        self.fc2 = nn.Linear(d2, num_features)
        self.d1 = d1
        self.d2 = d2
        
        self.num_features = num_features
        self.num_classes = num_classes
        
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # 2 fc layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # output as regression target
        return x
        
    def __str__(self):
        return "Net3-SAGEgcn(%d,%d)-fc(%d,%d)-fc(%d,%d)" % (self.num_features,self.d1,self.d1,
                                                        self.d2,self.d2,
                                                        self.num_features)

class Net4(torch.nn.Module, MyNet):
    def __init__(self, d1=300,d2=100,d3=10,num_features=1, num_classes=1):
        super(Net4, self).__init__()
        self.conv1 = GCNConv(num_features, d1 )
        self.conv2 = GCNConv(d1, d2)
        self.fc1 = nn.Linear(d2, d3)
        self.fc2 = nn.Linear(d3, num_features)
        self.d1 = d1
        self.d2 = d2
        self.d3 = d3

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # 2 fc layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        # output as regression target
        return x
        
    #def __str__(self):
    #    return "Net4-gcn(%d,%d)-fc(%d,%d)-fc(%d,%d)" % (dataset.num_features,self.d1,self.d1,
    #                                                    self.d2,self.d2,
    #                                                    dataset.num_features)

    
class Net5(torch.nn.Module, MyNet):
    def __init__(self, d1=90,d2=80,d3=50,d4=20,num_features=1, num_classes=1):
        super(Net5, self).__init__()
        self.conv1 = GCNConv(num_features, d1 )
        self.conv2 = GCNConv(d1, d2)
        self.fc1 = nn.Linear(d2, d3)
        self.fc2 = nn.Linear(d3, d4)
        self.fc3 = nn.Linear(d4, num_features)
        self.d1 = d1
        self.d2 = d2
        self.d3 = d3
        self.d4 = d4
        
    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        # 3 fc layers
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        # output as regression target
        return x
        
    #def __str__(self):
    #    #print(dir(self))
    #    #print()
    #    #print([attr+"-"+str(getattr(self,attr)) for attr in dir(self)])
    #    return "Net5-gcn(%d,%d)-fc(%d,%d)-fc(%d,%d)" % (dataset.num_features,self.d1,self.d1,
    #                                                   self.d2,self.d2,
    #                                                    dataset.num_features)


In [30]:
import networkx as nx
import time
import torch
from torch_geometric.data import DataLoader
import importlib
import torch
from torch_geometric.data import DataLoader
import networkx as nx
from torch_geometric.data import Data
import pickle
import math
import pandas as pd
from IPython.display import display, HTML
import os



def loadDataset(collection, name=None):
    try:
        # import datasets
        themodule = importlib.import_module("torch_geometric.datasets")
        # get the function corresponding to collection
        method_to_call = getattr(themodule, collection)
        if name:
            dataset = method_to_call(root='./data/'+str(collection), name=name)
            dataset.filename = name
            return dataset
        else:
            return method_to_call(root='./data/'+str(collection)) 
    except:
        # custom module
        method_to_call = globals()[collection]
       
        if name:
            
            dataset = method_to_call(root='./data/'+str(collection), name=name)
            dataset.filename = name
            return dataset
        else:
            return method_to_call(root='./data/'+str(collection)) 
        
def round_half_up(n, decimals=0):
    multiplier = 10 ** decimals
    return math.floor(n*multiplier + 0.5) / multiplier

def transformMask(mask):
    train_mask = []
    i = 0
    for pick in mask:
        if pick[0]==1:
            train_mask.append(i)
        i+=1
    return train_mask


def shuffleTrainTestMasks(data, trainpct = 0.7):
    ysize = list(data.y.size())[0]
    data.train_mask = torch.zeros(ysize,1, dtype=torch.long)
    data.train_mask[int(ysize*trainpct):] = 1
    data.train_mask = data.train_mask[torch.randperm(ysize)]
    data.test_mask = torch.ones(ysize,1, dtype=torch.long) - data.train_mask
    
    data.train_mask = transformMask(data.train_mask)
    data.test_mask = transformMask(data.test_mask)
  

def shuffleTrainTestValMasks(data, trainpct = 0.7, valpct = 0.2):

    ysize = list(data.y.size())[0]
    #print("total ", ysize)
    #print(" train ",int(ysize*trainpct)-int(ysize*trainpct*valpct))
    #print(" val ",int(ysize*trainpct*valpct))
    #print(" test ",int(ysize*(1- trainpct) ))
    data.train_mask = torch.zeros(ysize,1, dtype=torch.long)
    data.train_mask[:int(ysize*trainpct)] = 1
    data.train_mask = data.train_mask[torch.randperm(ysize)]
    #print(" train sum ",data.train_mask.sum())
    data.test_mask = torch.ones(ysize,1, dtype=torch.long) - data.train_mask
    #print(" test sum ",data.test_mask.sum())
    
    # transform to list of indexes
    data.train_mask = transformMask(data.train_mask)
    data.test_mask = transformMask(data.test_mask)
    
    data.val_mask = data.train_mask[:int(ysize*trainpct*valpct)]
    data.train_mask = data.train_mask[int(ysize*trainpct*valpct):]

    
    #print(data.train_mask)
    #print(data.val_mask)
    #print(data.test_mask)
    
    

def trainTestEval(dataset, epochs=1, batch_size=32, res_dict={}):
    global Net
    loader = DataLoader(dataset,  shuffle=False)
    i = 0
    #print(loader)
    #print(dir(loader))
    
    G = dataset.data
    #print(G)
    start = time.time()


    # 1.  prepare model
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    #print("using ",device)
    model = Net.to(device)  
    data = G.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    model.train()

    # 2.  create a train_mask, and a test_mask (val_mask for further experiments)
    #shuffleTrainTestMasks(data)
    #shuffleTrainTestValMasks(data)
    shuffleTrainTestMasks(data)

    # 3. train some epochs
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(data)
        loss = F.mse_loss(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        #if epoch % 25 == 0 :
        #    print("epoch-loss: ",epoch, loss)

    # 4. Model evaluation
    model.eval()
    #  classification in a multiclass setting
    #_, pred = model(data).max(dim=1)
    #correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
    #acc = correct / data.test_mask.sum().item()
    #print('Accuracy: {:.4f}'.format(acc))


    # regression 
    pred = model(data)
    #print("target: ",data.y[data.test_mask])
    #print("prediction: ",pred[data.test_mask])
    #print(pred[data.test_mask].type())
    #print(data.y[data.test_mask].type())
    
    # prepare the normalized mean root squared error
    t = data.y[data.test_mask]
    y = pred[data.test_mask]
    negatives = False
    if 0 > (sum(y<0)):
        negatives = True
    nrmse = torch.sum((t - y) ** 2)/len(data.test_mask)
    nrmse = nrmse.sqrt()
    #print("RMSE: ",nrmse)

    #m = torch.mean(t)
    #print("mean",m)
    #tmax = torch.max(t)
    #tmin = torch.min(t)
    #sd = tmax-tmin
    #print("sd",sd)
    #nrmse = (nrmse - m)/sd
    #print("NRMSE:",nrmse)


    endtime = time.time()
    #print("Total train-test time: "+str(endtime-start))

    # getting model hyper params
    hyperparams = ""
    model_ds = [elem+"="+str(getattr(Net, elem)) for elem in dir(Net) if elem.startswith("d") and len(elem)==2]
    hyperparams = "_".join(model_ds)
    theepochs = "epochs="+str(epochs)
    #dataset name
    thedataset = dataset.filename
    # basename
    thedataset = os.path.basename(thedataset)
    # extension
    thedataset = os.path.splitext(thedataset)[0]
    
    result = str(model)+" " \
            +hyperparams+" " \
            +theepochs+" " \
            +str(thedataset)+" " \
            +"nrmse="+str(round_half_up(nrmse.item(),3))+" " \
            +"time="+str(round_half_up(endtime-start,3) )  \
            +" negatives?"+str(negatives) \
            +"\n"
    
    res_dict[str(model)]={
                            #"model": str(model),
                             "hyperparams": hyperparams,
                             "dataset": str(thedataset),
                             "epochs": epochs,
                             "rmse":round_half_up(nrmse.item(),3),
                             "time":round_half_up(endtime-start,3),
                            "neg vals": negatives }
    
    with open("results.txt","a") as f:
        #print(dir(dataset))
        #f.write("\n")
        #print(result)
        f.write(result)
    
    del model

    #i+=1
    #if i==1:
    #    break
    
def reporting(d):
    df_original = pd.DataFrame(d)
    df = df_original.T
    #df.style.set_table_styles([ dict(selector='th', props=[('text-align', 'left')] ) ])
    # Assuming that dataframes df1 and df2 are already defined:
    display(df)
    #display(HTML(df.to_html()))
    #print(df)

In [31]:
class MyOwnDataset2():
    def __init__(self,  root, name, transform=None, pre_transform=None):
        f = open(name, 'rb')
        self.data = pickle.load(f) 
        #print(self.data.num_features)
        self.num_features = self.data.num_features
        self.num_classes = 1
        f.close()
        
        

In [32]:
def experimentBlock(dname):
    global Net
    
    dataset = loadDataset(
        collection='MyOwnDataset2', 
        name=dname)
    Net=Net1(d1=20, num_features=dataset.num_features, num_classes=dataset.num_classes)
    trainTestEval(dataset,  epochs=100, res_dict=res_dict)
    del Net

    dataset = loadDataset(
        collection='MyOwnDataset2', 
        name=dname)
    Net=Net1(d1=50, num_features=dataset.num_features, num_classes=dataset.num_classes)
    trainTestEval(dataset,  epochs=100, res_dict=res_dict)
    del Net

    dataset = loadDataset(
        collection='MyOwnDataset2', 
        name=dname)
    Net=Net1(d1=100, num_features=dataset.num_features, num_classes=dataset.num_classes)
    trainTestEval(dataset,  epochs=100, res_dict=res_dict)
    del Net

    dataset = loadDataset(
        collection='MyOwnDataset2', 
        name=dname)
    Net=Net2(d1=100,d2=15, num_features=dataset.num_features, num_classes=dataset.num_classes)
    trainTestEval(dataset,  epochs=100, res_dict=res_dict)
    del Net

    dataset = loadDataset(
        collection='MyOwnDataset2', 
        name=dname)
    Net=Net2(d1=50,d2=15, num_features=dataset.num_features, num_classes=dataset.num_classes)
    trainTestEval(dataset,  epochs=100, res_dict=res_dict)
    del Net

    dataset = loadDataset(
        collection='MyOwnDataset2', 
        name=dname)
    Net=Net2(d1=20,d2=10, num_features=dataset.num_features, num_classes=dataset.num_classes)
    trainTestEval(dataset,  epochs=100, res_dict=res_dict)
    del Net

    dataset = loadDataset(
        collection='MyOwnDataset2', 
        name=dname)
    Net=Net3(d1=50,d2=10, num_features=dataset.num_features, num_classes=dataset.num_classes)
    trainTestEval(dataset,  epochs=100, res_dict=res_dict)
    del Net

    dataset = loadDataset(
        collection='MyOwnDataset2', 
        name=dname)
    Net=Net4(d1=80,d2=40,d3=10, num_features=dataset.num_features, num_classes=dataset.num_classes)
    trainTestEval(dataset,  epochs=100, res_dict=res_dict)
    del Net

    dataset = loadDataset(
        collection='MyOwnDataset2', 
        name=dname)
    Net=Net5(d1=100,d2=50,d3=20,d4=5, num_features=dataset.num_features, num_classes=dataset.num_classes)
    trainTestEval(dataset,  epochs=100, res_dict=res_dict)
    del Net



# 2. Testing the models for node betweenness

In [33]:
res_dict={}

In [34]:
dname='../datasets/precomputed-pytorchg-betweenness/TUDataset_ENZYMES_11_nd.pickle'
experimentBlock(dname)
reporting(res_dict)
#dname='../datasets/precomputed-pytorchg-betweenness/er_1000_0_45_nb.pickle'
#experimentBlock(dname)

,dataset,epochs,hyperparams,neg vals,rmse,time
Net1-d1=20,TUDataset_ENZYMES_11_nd,100,d1=20,False,0.033,0.104
Net1-d1=50,TUDataset_ENZYMES_11_nd,100,d1=50,False,0.015,0.091
Net1-d1=100,TUDataset_ENZYMES_11_nd,100,d1=100,False,0.034,0.096
Net2-d1=100_d2=15,TUDataset_ENZYMES_11_nd,100,d1=100_d2=15,False,0,0.089
Net2-d1=50_d2=15,TUDataset_ENZYMES_11_nd,100,d1=50_d2=15,False,0.003,0.092
Net2-d1=20_d2=10,TUDataset_ENZYMES_11_nd,100,d1=20_d2=10,False,0.019,0.084
"Net3-SAGEgcn(1,50)-fc(50,10)-fc(10,1)",TUDataset_ENZYMES_11_nd,100,d1=50_d2=10,False,0.011,0.09
Net4-d1=80_d2=40_d3=10,TUDataset_ENZYMES_11_nd,100,d1=80_d2=40_d3=10,False,0.001,0.13
Net5-d1=100_d2=50_d3=20_d4=5,TUDataset_ENZYMES_11_nd,100,d1=100_d2=50_d3=20_d4=5,False,0.003,0.158


In [35]:
dname='../datasets/precomputed-pytorchg-betweenness/QM7b_5819_nd.pickle'
experimentBlock(dname)
reporting(res_dict)

,dataset,epochs,hyperparams,neg vals,rmse,time
Net1-d1=20,QM7b_5819_nd,100,d1=20,False,0.075,0.109
Net1-d1=50,QM7b_5819_nd,100,d1=50,False,0.042,0.12
Net1-d1=100,QM7b_5819_nd,100,d1=100,False,0.026,0.138
Net2-d1=100_d2=15,QM7b_5819_nd,100,d1=100_d2=15,False,0.001,0.135
Net2-d1=50_d2=15,QM7b_5819_nd,100,d1=50_d2=15,False,0.001,0.114
Net2-d1=20_d2=10,QM7b_5819_nd,100,d1=20_d2=10,False,0.006,0.094
"Net3-SAGEgcn(1,50)-fc(50,10)-fc(10,1)",QM7b_5819_nd,100,d1=50_d2=10,False,0.004,0.13
Net4-d1=80_d2=40_d3=10,QM7b_5819_nd,100,d1=80_d2=40_d3=10,False,0,0.186
Net5-d1=100_d2=50_d3=20_d4=5,QM7b_5819_nd,100,d1=100_d2=50_d3=20_d4=5,False,0.006,0.231


In [36]:
dname='../datasets/precomputed-pytorchg-betweenness/ws_30_3_0_1_nb.pickle'
experimentBlock(dname)
reporting(res_dict)

,dataset,epochs,hyperparams,neg vals,rmse,time
Net1-d1=20,ws_30_3_0_1_nb,100,d1=20,False,0.867,0.103
Net1-d1=50,ws_30_3_0_1_nb,100,d1=50,False,0.74,0.111
Net1-d1=100,ws_30_3_0_1_nb,100,d1=100,False,0.854,0.113
Net2-d1=100_d2=15,ws_30_3_0_1_nb,100,d1=100_d2=15,False,0.997,0.106
Net2-d1=50_d2=15,ws_30_3_0_1_nb,100,d1=50_d2=15,False,0.934,0.1
Net2-d1=20_d2=10,ws_30_3_0_1_nb,100,d1=20_d2=10,False,0.893,0.089
"Net3-SAGEgcn(1,50)-fc(50,10)-fc(10,1)",ws_30_3_0_1_nb,100,d1=50_d2=10,False,0.816,0.104
Net4-d1=80_d2=40_d3=10,ws_30_3_0_1_nb,100,d1=80_d2=40_d3=10,False,0.713,0.159
Net5-d1=100_d2=50_d3=20_d4=5,ws_30_3_0_1_nb,100,d1=100_d2=50_d3=20_d4=5,False,0.835,0.192


In [37]:

dname='../datasets/precomputed-pytorchg-betweenness/ws_1000_10_0_1_nb.pickle'
experimentBlock(dname)
reporting(res_dict)

,dataset,epochs,hyperparams,neg vals,rmse,time
Net1-d1=20,ws_1000_10_0_1_nb,100,d1=20,False,0.134,0.564
Net1-d1=50,ws_1000_10_0_1_nb,100,d1=50,False,0.07,1.401
Net1-d1=100,ws_1000_10_0_1_nb,100,d1=100,False,0.07,3.204
Net2-d1=100_d2=15,ws_1000_10_0_1_nb,100,d1=100_d2=15,False,0.074,2.271
Net2-d1=50_d2=15,ws_1000_10_0_1_nb,100,d1=50_d2=15,False,0.072,1.357
Net2-d1=20_d2=10,ws_1000_10_0_1_nb,100,d1=20_d2=10,False,0.114,0.517
"Net3-SAGEgcn(1,50)-fc(50,10)-fc(10,1)",ws_1000_10_0_1_nb,100,d1=50_d2=10,False,0.07,1.808
Net4-d1=80_d2=40_d3=10,ws_1000_10_0_1_nb,100,d1=80_d2=40_d3=10,False,0.07,2.969
Net5-d1=100_d2=50_d3=20_d4=5,ws_1000_10_0_1_nb,100,d1=100_d2=50_d3=20_d4=5,False,0.071,3.553


In [38]:
dname='../datasets/precomputed-pytorchg-betweenness/ba_1000_5_nb.pickle'
experimentBlock(dname)
reporting(res_dict)

,dataset,epochs,hyperparams,neg vals,rmse,time
Net1-d1=20,ba_1000_5_nb,100,d1=20,False,0.389,0.587
Net1-d1=50,ba_1000_5_nb,100,d1=50,False,0.155,1.465
Net1-d1=100,ba_1000_5_nb,100,d1=100,False,0.226,2.673
Net2-d1=100_d2=15,ba_1000_5_nb,100,d1=100_d2=15,False,0.138,2.599
Net2-d1=50_d2=15,ba_1000_5_nb,100,d1=50_d2=15,False,0.162,1.374
Net2-d1=20_d2=10,ba_1000_5_nb,100,d1=20_d2=10,False,0.169,0.536
"Net3-SAGEgcn(1,50)-fc(50,10)-fc(10,1)",ba_1000_5_nb,100,d1=50_d2=10,False,0.153,1.879
Net4-d1=80_d2=40_d3=10,ba_1000_5_nb,100,d1=80_d2=40_d3=10,False,0.161,2.791
Net5-d1=100_d2=50_d3=20_d4=5,ba_1000_5_nb,100,d1=100_d2=50_d3=20_d4=5,False,0.243,3.699


In [39]:
dname='../datasets/precomputed-pytorchg-betweenness/ba_100_5_nb.pickle'
experimentBlock(dname)
reporting(res_dict)

,dataset,epochs,hyperparams,neg vals,rmse,time
Net1-d1=20,ba_100_5_nb,100,d1=20,False,0.198,0.136
Net1-d1=50,ba_100_5_nb,100,d1=50,False,0.214,0.185
Net1-d1=100,ba_100_5_nb,100,d1=100,False,0.229,0.256
Net2-d1=100_d2=15,ba_100_5_nb,100,d1=100_d2=15,False,0.184,0.246
Net2-d1=50_d2=15,ba_100_5_nb,100,d1=50_d2=15,False,0.14,0.175
Net2-d1=20_d2=10,ba_100_5_nb,100,d1=20_d2=10,False,0.182,0.127
"Net3-SAGEgcn(1,50)-fc(50,10)-fc(10,1)",ba_100_5_nb,100,d1=50_d2=10,False,0.148,0.192
Net4-d1=80_d2=40_d3=10,ba_100_5_nb,100,d1=80_d2=40_d3=10,False,0.184,0.323
Net5-d1=100_d2=50_d3=20_d4=5,ba_100_5_nb,100,d1=100_d2=50_d3=20_d4=5,False,0.16,0.398


In [40]:
dname='../datasets/precomputed-pytorchg-betweenness/Planetoid_Cora_1_nd.pickle'
experimentBlock(dname)
reporting(res_dict)

,dataset,epochs,hyperparams,neg vals,rmse,time
Net1-d1=20,Planetoid_Cora_1_nd,100,d1=20,False,0.582,0.945
Net1-d1=50,Planetoid_Cora_1_nd,100,d1=50,False,0.351,2.037
Net1-d1=100,Planetoid_Cora_1_nd,100,d1=100,False,0.324,3.505
Net2-d1=100_d2=15,Planetoid_Cora_1_nd,100,d1=100_d2=15,False,0.324,3.465
Net2-d1=50_d2=15,Planetoid_Cora_1_nd,100,d1=50_d2=15,False,0.319,1.678
Net2-d1=20_d2=10,Planetoid_Cora_1_nd,100,d1=20_d2=10,False,0.265,0.927
"Net3-SAGEgcn(1,50)-fc(50,10)-fc(10,1)",Planetoid_Cora_1_nd,100,d1=50_d2=10,False,0.326,2.543
Net4-d1=80_d2=40_d3=10,Planetoid_Cora_1_nd,100,d1=80_d2=40_d3=10,False,0.326,4.325
Net5-d1=100_d2=50_d3=20_d4=5,Planetoid_Cora_1_nd,100,d1=100_d2=50_d3=20_d4=5,False,0.377,5.207


In [41]:
dname='../datasets/precomputed-pytorchg-betweenness/KarateClub_1_nd.pickle'
experimentBlock(dname)
reporting(res_dict)

,dataset,epochs,hyperparams,neg vals,rmse,time
Net1-d1=20,KarateClub_1_nd,100,d1=20,False,0.462,0.108
Net1-d1=50,KarateClub_1_nd,100,d1=50,False,0.351,0.108
Net1-d1=100,KarateClub_1_nd,100,d1=100,False,0.357,0.123
Net2-d1=100_d2=15,KarateClub_1_nd,100,d1=100_d2=15,False,0.538,0.108
Net2-d1=50_d2=15,KarateClub_1_nd,100,d1=50_d2=15,False,0.456,0.099
Net2-d1=20_d2=10,KarateClub_1_nd,100,d1=20_d2=10,False,0.613,0.102
"Net3-SAGEgcn(1,50)-fc(50,10)-fc(10,1)",KarateClub_1_nd,100,d1=50_d2=10,False,0.454,0.118
Net4-d1=80_d2=40_d3=10,KarateClub_1_nd,100,d1=80_d2=40_d3=10,False,0.566,0.177
Net5-d1=100_d2=50_d3=20_d4=5,KarateClub_1_nd,100,d1=100_d2=50_d3=20_d4=5,False,0.377,0.211
